In [1]:
import torch
from torchvision import models
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image

* 파이토치 기본 자료구조인 텐서의 이해
* 텐서를 인덱스로 접근해서 연산하기
* 다차원 배열 넘파이와 연계해서 다루기
* 성능 개선을 위해 GPU로 연산 처리

심층신경망의 중간단계는 입력값의 특징을 잡아내는 부동소수점 수의 모음인 동시에 신경망에서 입력이 최종적으로 출력으로 표현되는 방법을 기술하기 위한 수단으로 데이터구조를 잡아낸다.

중간 표현값은 부동소수좀으로 이루어져있고, 입력을 특징짓는 동시에 데이터구조를 잡아내어 신경망 출력과 어떻게 매핑되는지를 설명하는 데 중요한 역할을 한다.

이러한 중간 표현값은 입력과 이전 층의 뉴련이 가진 가중치를 조합한 결과이다. 

* 텐서 : 임의의 차원을 가진 벡터나 행렬의 일반화된 개념 = 다차원 배열 = N차원 데이터

In [3]:
torch.ones(3)

tensor([1., 1., 1.])

In [4]:
torch.zeros(6)

tensor([0., 0., 0., 0., 0., 0.])

In [5]:
# 생성자에 파이썬 리스트를 넘겨도 된다
points = torch.tensor([4.0, 1.0, 2.0, 1.0, 5.0])
points

tensor([4., 1., 2., 1., 5.])

In [6]:
points = torch.tensor([ [4.0, 1.0], [5.0, 3.0], [2.0, 1.0] ])
print(points)
print(points.shape)

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
torch.Size([3, 2])


In [7]:
torch.zeros(3,2)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

### 이름이 있는 텐서

In [14]:
img_t = torch.randn(3, 5, 5) # 채널크기, 행크기, 열크기
weights = torch.tensor([0.2162, 0.7234, 0.3234])
print(weights.size())
batch_t = torch.randn(2, 1, 5, 5) # 배치크기, 채널크기, 행 크기, 열 크기
batch_t

torch.Size([3])


tensor([[[[ 0.1736, -0.4527,  0.0623, -0.2101, -2.1567],
          [-0.8765,  0.1159, -1.8345,  0.0051, -2.7995],
          [-1.1394,  1.5173,  1.1913,  0.5965, -1.3398],
          [ 0.5310,  1.7657, -3.2736, -0.2540, -0.4868],
          [ 1.2080,  0.9093,  1.1096, -0.1413, -0.2806]]],


        [[[ 0.5207, -1.7061,  0.2460,  0.0794, -0.1606],
          [ 1.8150,  0.1819,  0.4458, -0.4629,  1.0726],
          [-0.0621, -0.8098,  0.1403, -0.3068, -0.5323],
          [ 0.7245,  0.1004, -0.8556, -0.2167, -0.0162],
          [-1.5043, -0.7533,  0.0816, -0.6315, -0.2560]]]])

In [16]:
# 채널이 3
batch_t = torch.randn(2, 3, 5, 5)


# RGB 채널은 0번 혹은 1번째 차원에 있다. 두 경우 모두 뒤에서 3번째 차원이므로 RGB 채널은 -3번 차원에 있는 것으로 일반화할 수 있다.

# 배치 유무에 상관없이 평균을 구할 수 있다.
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

### 브로드캐스팅 

파이토치는 동일한 차원 정보의 텐서끼리 연산할 수 있고, 각 차원의 길이가 1인 텐서도 가능하다.

혹은 길이가 1인 차원을 알아서 늘려주기도 한다.

In [18]:
# (2, 3, 5, 5) 차원을 가진 batch_t 를 (3, 1 ,1) 차원의 unsqueezed_weights 로 곱하면 (2, 3, 5, 5) 차원이 된다.
# 채널 정보를 가진, 뒤에서 세 번째 차원에 있는 값에 대한 합을 구할 수 있다.

unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1) # (3) -> (3, 1) -> (3, 1, 1)


img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)


img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)

batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

# 서로 다른 차원을 가진 텐서는 원래 곱할 수 없는데, 파이토치가 편의상 동일한 차원으로 알아서 맞춰준 후 곱해주므로 (2, 3, 5, 5) 차원이 된다.

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [21]:
# tensor나 rand 같은 텐서 팩토리 함수에 이름으로 사용할 문자열 리스트를 names 인자로 전달할 수 있다.
weights_named = torch.tensor([0.2142, 0.3234, 0.4321], names=['channels'])
weights_named


tensor([0.2142, 0.3234, 0.4321], names=('channels',))

In [22]:
# 텐서끼리의 연산은 먼저 각 차원의 크기가 같은지, 한쪽이 1이고 다른쪽으로 브로드캐스팅 될 수 있는지 확인해야 한다. 

# 이름이 지정되어 있다면 파이토치가 우리를 대신해 알아서 체크해준다.

In [24]:
# align 함수 : 빠진 채원을 채우고, 존재하는 차원을 올바른 순서로 바꿔준다.

img_named = img_t.refine_names(..., 'channels', 'rows', 'cols')

weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'cols'))

### 텐서 요소 타입

* 파이썬에서 숫자는 객체다. -> 백만 개가 넘어가면 비효율적이다.
* 파이썬에서 리스트는 연속된 객체의 컬렉션이다. -> 두 벡터의 내적을 효율적으로 수행하는 연산이 없다.
* 파이썬 인터프리터는 최적화를 거치는 컴파일된 코드보다 느리다. -> C같은 저수준 컴파일을 통해 최적화된 바이너리 코드가 훨씬 빠르다.

-> 파이토치 텐서같이 전용 데이터 구조를 만든 후 숫자 데이터 연산은 저수준 언어로 효율을 높인다. 성능 최적화르 위해 텐서 내의 모든 객체는 같은 타입의 숫자여야 한다.

In [ ]:
# dtype 으로 숫자 타입 지정하기 : 텐서의 기본 데이터 타입은 32비트 부동소수점(float32) 이다.
# float 부동소수점
# int 부호 있는 정수
# uint 부호 없는 정수
# bool 불리언



# 변환이 필요한 경우 to 메소드 이용

double_points = torch.zeros(10, 2).to(torch.double)


# 여러 타입을 가진 입력들이 연산을 거치며 서로 섞일 때, 자동으로 제일 큰 타입으로 만들어진다.

# 텐서 API 

파이토치가 제공하는 텐서 연산

In [25]:
# transpose

a = torch.ones(3,2)
a_t = torch.transpose(a, 0, 1)
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [26]:
# 텐서 메소드 이용
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [27]:
# 텐서의 size = 넘파이의 shape
# 오프셋 : 텐서의 첫번째 요소를 가리키는 색인 값
# 스트라이드 : 각 차원에서 다음 요소를 가리키고 싶을 때 실제 저장 공간상에서 몇 개의 요소를 건너뛰어야 하는지 알려주는 숫자


In [32]:
# 전치 transpose(), t()

points = torch.tensor( [[4.0, 1.0], [1.0, 3.0], [3.0, 9.0]] )
points

tensor([[4., 1.],
        [1., 3.],
        [3., 9.]])

In [34]:
points_t = points.t()
points_t

tensor([[4., 1., 3.],
        [1., 3., 9.]])

In [35]:
id(points.storage() == id(points_t.storage())) # 두 텐서가 같은 공간을 가리키고 있는가? True

/var/folders/8l/9jsw0prj36ngswky608_l6rc0000gn/T/ipykernel_57567/687426748.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  id(points.storage() == id(points_t.storage()))


4334755728